In [1]:
from __future__ import print_function
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np
import os

In [2]:
######################################
######### Necessary Flags ############
######################################

num_classes = 10
batch_size = 128
num_epochs = 10

In [3]:
##########################################
######## Learning rate flags #############
##########################################
initial_learning_rate = 0.001
learning_rate_decay_factor = 0.95
num_epochs_per_decay = 1

In [4]:
#########################################
########## status flags #################
#########################################
is_training = False
fine_tuning = False
online_test = True
allow_soft_placement = True
log_device_placement = False

In [5]:
##########################################
####### Load and Organize Data ###########
##########################################
'''
In this part the input must be prepared.

   1 - The MNIST data will be downloaded.
   2 - The images and labels for both training and testing will be extracted.
   3 - The prepared data format(?,784) is different by the appropriate image shape(?,28,28,1) which needs
        to be fed to the CNN architecture. So it needs to be reshaped.

'''

# Download and get MNIST dataset(available in tensorflow.contrib.learn.python.learn.datasets.mnist)
# It checks and download MNIST if it's not already downloaded then extract it.
# The 'reshape' is True by default to extract feature vectors but we set it to false to we get the original images.
mnist = input_data.read_data_sets("MNIST_data/", reshape=True, one_hot=True)
train_data = mnist.train.images
train_label = mnist.train.labels
test_data = mnist.test.images
test_label = mnist.test.labels

# # The 'input.provide_data' is provided to organize any custom dataset which has specific characteristics.
# data = input.provide_data(mnist)

# Dimentionality of train
dimensionality_train = train_data.shape

# Dimensions
num_train_samples = dimensionality_train[0]
num_features = dimensionality_train[1]

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [11]:
graph = tf.Graph()
with graph.as_default():
    global_step = tf.Variable(0, name="global_step", trainable=False)
    decay_steps = int(num_train_samples / batch_size * num_epochs_per_decay)
    learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step, decay_steps, learning_rate_decay_factor, staircase=True, name="exponential_decay_learning_rate")

    #######
    ### Defining place holders
    #######

    image_place = tf.placeholder(tf.float32, shape=([None, num_features]), name='image')
    label_place = tf.placeholder(tf.float32, shape=([None, num_classes]), name='gt')
    dropout_param = tf.placeholder(tf.float32)


    #####
    ### MODEL(MPL with two hidden layer)
    ####

    net = tf.contrib.layers.fully_connected(inputs=image_place, num_outputs=250, scope='fc-1')
    net = tf.contrib.layers.fully_connected(inputs=net, num_outputs=250, scope='fc-2')
    logits_pre_softmax = tf.contrib.layers.fully_connected(inputs=net, num_outputs=num_classes, scope='fc-3')
    softmax_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_pre_softmax, labels=label_place))
    #tf.argmax返回的是vector中的最大值的索引号，如果vector是一个向量，那就返回一个值，如果是一个矩阵，那就返回一个向量；axis=1是行为单位，返回最大列的索引，这里就是判断是否一致。
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(logits_pre_softmax,1), tf.argmax(label_place,1)), tf.float32))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

    with tf.name_scope('train_scope'):
        #grads = optimizer.compute_gradients(softmax_loss)
        #train_op = optimizer.apply_gradients(grads, global_step=global_step)
        train_op = optimizer.minimize(softmax_loss, global_step=global_step)
    
    # Summaries for loss and accuracy
    tf.summary.scalar("loss", softmax_loss, collections=['train', 'test'])
    tf.summary.scalar("accuracy", accuracy, collections=['train', 'test'])
    tf.summary.scalar("global_step", global_step, collections=['train'])
    tf.summary.scalar("learning_rate", learning_rate, collections=['train'])
    
    # Merge all summaries together.
    summary_train_op = tf.summary.merge_all('train')
    summary_test_op = tf.summary.merge_all('test')

    session_conf = tf.ConfigProto(allow_soft_placement=allow_soft_placement, log_device_placement=log_device_placement)
    sess = tf.Session(graph=graph, config=session_conf)

    with sess.as_default():
        sess.run(tf.global_variables_initializer())
        for epoch in range(num_epochs):
            total_batch_training = int(train_data.shape[0] / batch_size)

            for batch_num in range(total_batch_training):
                start_idx = batch_num * batch_size
                end_idx = (batch_num + 1) * batch_size

                train_batch_data, train_batch_label = train_data[start_idx:end_idx], train_label[start_idx:end_idx]

                batch_loss,_,training_step = sess.run(
                    [softmax_loss, train_op, global_step], 
                    feed_dict={image_place:train_batch_data, 
                        label_place:train_batch_label, 
                        dropout_param:0.5})

            print("Epoch #" + str(epoch+1) + ", Train Loss=" + "{:.3f}".format(batch_loss))

            if online_test:
                # WARNING: In this evaluation the whole test data is fed. In case the test data is huge this implementation
                #          may lead to memory error. In presense of large testing samples, batch evaluation on testing is
                #          recommended as in the training phase.
                test_accuracy_epoch, test_summaries = sess.run(
                    [accuracy, summary_test_op],
                    feed_dict={image_place: test_data,
                               label_place: test_label,
                               dropout_param: 1.})
                print("Test Accuracy= " + \
                      "{:.4f}".format(test_accuracy_epoch))

                ###########################################################
                ########## Write the summaries for test phase #############
                ###########################################################

                # Returning the value of global_step if necessary
                current_step = tf.train.global_step(sess, global_step)


        # Evaluation of the model
        total_test_accuracy = sess.run(accuracy, feed_dict={
            image_place: test_data,
            label_place: test_label,
            dropout_param: 1.})

        print("Final Test Accuracy is %.2f" % total_test_accuracy)


Epoch #1, Train Loss=0.524
Test Accuracy= 0.7727
Epoch #2, Train Loss=0.515
Test Accuracy= 0.7792
Epoch #3, Train Loss=0.511
Test Accuracy= 0.7841
Epoch #4, Train Loss=0.510
Test Accuracy= 0.7859
Epoch #5, Train Loss=0.508
Test Accuracy= 0.7852
Epoch #6, Train Loss=0.506
Test Accuracy= 0.7830
Epoch #7, Train Loss=0.505
Test Accuracy= 0.7821
Epoch #8, Train Loss=0.504
Test Accuracy= 0.7890
Epoch #9, Train Loss=0.504
Test Accuracy= 0.7886
Epoch #10, Train Loss=0.504
Test Accuracy= 0.7879
Final Test Accuracy is 0.79
